In [3]:
import torch.nn as nn
import torch


In [1]:
#X:batchsize*num*input_size
#ht-1 :batchsize*num*hidden_size

In [2]:
nn.init.xavier_uniform_(torch.randn(2,3,4))

NameError: name 'nn' is not defined

In [ ]:
class GRUCell(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(GRUCell, self).__init__()

        self.input_size=input_size
        self.hidden_size=hidden_size

        #更新门参数
        self.Wz_x=nn.Parameter(torch.randn(input_size,hidden_size))
        self.Wz_h=nn.Parameter(torch.randn(hidden_size,hidden_size))
        self.bz=nn.Parameter(torch.zeros(hidden_size))
        # 重置门参数
        self.Wr_x=nn.Parameter(torch.randn(input_size,hidden_size))
        self.Wr_h=nn.Parameter(torch.randn(hidden_size,hidden_size))
        self.br=nn.Parameter(torch.zeros(hidden_size))

        # 候选隐状态参数
        self.Wh_x=nn.Parameter(torch.randn(input_size,hidden_size))
        self.Wh_h=nn.Parameter(torch.randn(hidden_size,hidden_size))
        self.bh=nn.Parameter(torch.zeros(hidden_size))



    def forward(self,X,hidden_state_pre):
        """
        输入 X batch_size*input_size, ht-1  batch_size*hidden_size
        """
        # 1、计算更新门
        z_gate=torch.sigmoid(torch.mm(X,self.Wz_x)+torch.mm(hidden_state_pre,self.Wz_h)+self.bz )
        #2、计算重置门
        r_gate=torch.sigmoid(torch.mm(X,self.Wr_x)+torch.mm(hidden_state_pre,self.Wr_h)+self.br )

        # 3、计算候选隐状态
        h_tmp=torch.tanh(torch.mm(X,self.Wh_x)+torch.mm((hidden_state_pre*r_gate),self.Wh_h)+self.bh )

        # 4、计算当前步隐藏层

        h=z_gate*hidden_state_pre+(1-z_gate)*h_tmp

        return h


In [ ]:
# qkv q与k点积/d 再与v相乘

## 缩放点积注意力

# q batch_size*查询个数*d
# k batch_size*键值对个数*d
# v batch*键值对个数*hidden_size
class DotAttention(nn.Module):
    def __init__(self,dropout=0.1):
        super(DotAttention, self).__init__()

        self.dropout=nn.Dropout(dropout)
    def forward(self,q,k,v,mask):
        k=k.transpose(-1,-2)
        d_size=q.shape[-1]
        attention_score=torch.bmm(q,k)/torch.sqrt(torch.tensor(d_size))
        attention_score[~mask]=torch.tensor(-1e6)
        attention_weights=torch.softmax(attention_score,dim=-1)
        return torch.bmm(self.dropout(attention_weights),v)

In [ ]:
# 掩蔽softmax实现
def sequence_mask(X,valid_len, value=0):
    """X必须为2维batch_size* num_steps，valid_len必须为1维batch
       意味着 第i个batch_size的序列有效长度为 valid_len[i]
    """
    if X.dim() != 2 or valid_len.dim() != 1:
        raise ValueError('Expect 2d tensor')
    # 获取num_steps长度
    maxlen = X.shape[1]
    # 生成顺序序列，用于与valid_len比较
    mask = torch.unsqueeze(torch.arange(0, maxlen, dtype=torch.long),dim=0)
    # 判断需要掩码的部分，这里必须要保证mask和vilid_len形状相等 否则会出问题
    mask = (mask<valid_len.unsqueeze(1).repeat(1, maxlen))
    X[~mask] = value
    return X

def masked_softmax(X,valid_lens=None):
    """通过在最后一个轴上掩蔽元素来执行softmax操作
    注意：X是三维的 一般是 B Q K-V数量
     掩码的目的在于 避免Q关注到无关的或者禁止的K-V
    """
    if valid_lens is None:
        return F.softmax(X, dim=-1)
    shape=X.shape
    # valid_lens可以是1d或者2d 如果是1d则必须长为B 如果是2d则必须是B*Q
    if valid_lens.dim() == 1:
        # 如果是1d则扩展成2d 形状B Q
        valid_lens = valid_lens.unsqueeze(1).repeat(1, shape[1])
    # 因为sequence函数要求的X是2d valid_lens是1d
    X=X.reshape(-1,shape[-1])
    valid_lens=valid_lens.reshape(-1)
    X_mask=sequence_mask(X, valid_lens,value=-1e6)
    # 完事之后还要把输出的形状转为X原本的形状
    return F.softmax(X_mask, dim=-1).reshape(shape)

In [ ]:
#@save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量

    shape = X.shape

    valid_lens = valid_lens.reshape(-1)
    # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmaxb输出为0
    X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                          value=-1e6)
    return nn.functional.softmax(X.reshape(shape), dim=-1)

In [ ]:
mask = torch.unsqueeze(torch.arange(0, maxlen, dtype=torch.long),dim=0)
# 判断需要掩码的部分，这里必须要保证mask和vilid_len形状相等 否则会出问题
mask = (mask<valid_len.unsqueeze(1).repeat(1, maxlen))

In [21]:
#@save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量 b,query_num,key_num ，valid_lens:2D张量 b,query_num
    x_shape=X.shape
    X=X.reshape(-1,x_shape[-1]) # b*query_num,key_num
    max_len=X.shape[-1]
    valid_lens=valid_lens.reshape(-1)# b*query_num
    mask=torch.unsqueeze(torch.arange(0,max_len,dtype=torch.long),dim=0)# 1,key_num

    #此时mask b*query_num key_num
    mask=(mask<torch.unsqueeze(valid_lens,-1))
    X[~mask]=-1e6

    return F.softmax(X, dim=-1).reshape(x_shape)
    # mask = (mask<valid_len.unsqueeze(1))


In [22]:
X=torch.zeros(1,3,4)
valid_lens=torch.tensor([[1,2,3]])

In [23]:
masked_softmax(X,valid_lens)

tensor([[       0., -1000000., -1000000., -1000000.],
        [       0.,        0., -1000000., -1000000.],
        [       0.,        0.,        0., -1000000.]])

In [25]:
def sequence_mask(X,valid_len, value=0):
    """X必须为2维batch_size* num_steps，valid_len必须为1维batch
       意味着 第i个batch_size的序列有效长度为 valid_len[i]
    """
    if X.dim() != 2 or valid_len.dim() != 1:
        raise ValueError('Expect 2d tensor')
    # 获取num_steps长度
    maxlen = X.shape[1]
    # 生成顺序序列，用于与valid_len比较
    mask = torch.unsqueeze(torch.arange(0, maxlen, dtype=torch.long),dim=0)
    # 判断需要掩码的部分，这里必须要保证mask和vilid_len形状相等 否则会出问题
    mask = (mask<valid_len.unsqueeze(1).repeat(1, maxlen))
    X[~mask] = value
    return X

def sequence_mask(X,valid_len, value=0):
    """X必须为2维batch_size* num_steps，valid_len必须为1维batch
       意味着 第i个batch_size的序列有效长度为 valid_len[i]
    """
    if X.dim() != 2 or valid_len.dim() != 1:
        raise ValueError('Expect 2d tensor')
    # 获取num_steps长度
    maxlen = X.shape[1]
    # 生成顺序序列，用于与valid_len比较
    mask = torch.unsqueeze(torch.arange(0, maxlen, dtype=torch.long),dim=0)
    # 判断需要掩码的部分，这里必须要保证mask和vilid_len形状相等 否则会出问题
    mask = (mask<valid_len.unsqueeze(1))
    X[~mask] = value
    return X

In [26]:
sequence_mask(X,valid_lens)

ValueError: Expect 2d tensor

In [25]:
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    def __init__(self,q_size,k_size,v_size,hidden_size,num_heads,dropout=0.1):
        super(MultiHeadAttention,self).__init__()
        
        self.num_heads=num_heads
        self.W_q=nn.Linear(q_size,hidden_size,bias=False)
        self.W_k=nn.Linear(k_size,hidden_size,bias=False)
        self.W_v=nn.Linear(v_size,hidden_size,bias=False)
        
        self.W_o=nn.Linear(hidden_size,hidden_size,bias=False)
        self.dropout=nn.Dropout(dropout)
		
    def forward(self,q,k,v,mask):
        """
        mask与q shape相等
        """
        query=self.W_q(q)
        key=self.W_q(k)
        value=self.W_q(v)
        
        q_shape=query.shape
        k_shape=key.shape
        
        query=query.reshape(q_shape[0],q_shape[1],self.num_heads,-1).permute(0,2,1,3)
        key=key.reshape(k_shape[0],k_shape[1],self.num_heads,-1).permute(0,2,1,3)
        value=value.reshape(k_shape[0],k_shape[1],self.num_heads,-1).permute(0,2,1,3)
        
        query_m=query.reshape(-1,query.shape[2],query.shape[3])
        key_m=key.reshape(-1,key.shape[2],key.shape[3])
        value_m=value.reshape(-1,value.shape[2],value.shape[3])
        
        ############缩放点积注意力###############
        d=query_m.shape[-1]
        
        attention_score=torch.bmm(query_m,key_m.permute(0,2,1))/math.sqrt(d)
        
        if mask:
            mask=mask.repeat_interval(self.num_heads,dim=0)
        
            attention_score[~mask]=torch.tensor(-1e9)
        
        attention_weight=torch.nn.functional.softmax(attention_score,dim=-1)## b*num_heads,q_num,k_num
        
        attention_result=torch.bmm(self.dropout(attention_weight),value_m)
        
        attention_result=attention_result.reshape(q_shape[0],self.num_heads,q_shape[1],-1).permute(0,2,1,3).reshape(q_shape[0],q_shape[1],-1)
        
        return self.W_o(attention_result)

In [26]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.eval()

MultiHeadAttention(
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [27]:
batch_size, num_queries = 2, 4
num_kvpairs, valid_lens =  6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y,mask=None).shape

torch.Size([2, 4, 100])